In [ ]:
participants = [
    {"name": "Mario", "email": "mario.rossi@mail.com"},
    {"name": "Luigi", "email": "luigi.ferrari@mail.com"},
]


names = [p["name"] for p in participants]

if len(names) != len(set(names)):
    raise ValueError("❌ Duplicated names in the participants")


emails = [p["email"] for p in participants]

if len(emails) != len(set(emails)):
    raise ValueError("❌ Duplicated emails in the participants")

In [42]:
import random

def extract_secret_santa(participants):
    
    alreadyExtracted = set()
    pairings = {}

    for p in participants:
        
        extractionList = [x for x in participants if x["name"] not in alreadyExtracted and x != p]
    
        giver = p
        receiver = random.choice(extractionList)

        alreadyExtracted.add(receiver["name"])

        pairings[giver["email"]] = {"giver": giver["name"], "receiver": receiver["name"]}

    return pairings

In [ ]:
import smtplib
from email.message import EmailMessage
from email.utils import make_msgid

def send_email(receiver_email, giver_name, receiver_name):
    msg = EmailMessage()
    msg["Subject"] = "🎅 Secret Santa 🎁"
    msg["From"] = "Babbo Natale"
    msg["To"] = receiver_email

    image_cid = make_msgid(domain="secretsanta")
    
    html = f"""
    <html>
        <body style="font-family: Arial;">
            <h2>Ciao {giver_name} 🎄</h2>

            <p>Sei stato scelto come <b>Secret Santa</b> per:</p>

            <h1 style="color: red;">🎁 {receiver_name} 🎁</h1>

            <img src="cid:{image_cid[1:-1]}" width="300">

            <p>Mi raccomando, non dirlo a nessuno 😉</p>
            <p><i>Buon Natale!</i></p>
        </body>
    </html>
    """

    msg.add_alternative(html, subtype="html")

    with open("Pictures/"+ receiver_name + ".png", "rb") as img:
        msg.get_payload()[0].add_related(
            img.read(),
            maintype="image",
            subtype="png",
            cid=image_cid
        )

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login("yourSantaEmail@mail.com", "Password")
        server.send_message(msg)


In [ ]:
pairings = extract_secret_santa(participants)

for email, info in pairings.items():
    send_email(
        receiver_email=email,
        giver_name=info["giver"],
        receiver_name=info["receiver"]
    )